## AUPR for evaluating taxonomic profiles in the benchmarking work 
* Dec. 22, 2020

This notebook is intented to illustrate how to calculate the AUPRC based on an expected abundance profile and observed one. 

In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import auc
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
%matplotlib inline 

### precision and recall in general
* Precision: the ratio tp / (tp + fp) where tp is the number of true positives and fp the number of false positives. The precision is intuitively the ability of the classifier not to label as positive a sample that is negative.

* Recall: the ratio tp / (tp + fn) where tp is the number of true positives and fn the number of false negatives. The recall is intuitively the ability of the classifier to find all the positive samples.


### precision and recall in the context of evaluating the performance of taxonomic profiles

* Precision: the proportion of true classified taxa over all classified taxa. 
* Recall: the proportion of correctly classified abundances over all truth abundances


### the updated function for calculating precision recall curve
* we modified the recall=1.0 with the higest observed recall in the resulting recall vector,
* It is also supported by the CELL paper, where they "set the precision to 0 for the highest observed recall onward" (https://www.cell.com/cms/10.1016/j.cell.2019.07.010/attachment/3a60e4f3-3b48-4909-8c4c-c3da3fd46267/mmc1.pdf)

In [14]:
def to_labels(abd, threshold):
    return (abd>threshold).astype('int')

def updated_prc(expected_abd, observed_abd, thresholds=[0, 0.1, 0.001]):
    bi_expected_abd=np.array(expected_abd!=0, dtype=int)
    precision, recall, _ = precision_recall_curve(bi_expected_abd, observed_abd)
    precision[0]=max(precision[1:])
    recall[0]=max(recall[1:])
    aupr=auc(recall, precision)
    f1_scores = [f1_score(bi_expected_abd, to_labels(observed_abd, i)) for i in thresholds]
    precision_scores = [precision_score(bi_expected_abd, to_labels(observed_abd, i)) for i in thresholds]
    recall_scores = [recall_score(bi_expected_abd, to_labels(observed_abd, i)) for i in thresholds]
    return np.append(np.concatenate((precision_scores, recall_scores, f1_scores)), aupr)

Example:

In [16]:
expected_abd=np.array([0, 0.4, 0.3, 0, 0.2, 0.1, 0])
observed_abd=np.array([0.1, 0, 0.2, 0, 0.7, 0, 0])
updated_prc(expected_abd, observed_abd)

array([0.66666667, 1.        , 0.66666667, 0.5       , 0.5       ,
       0.5       , 0.57142857, 0.66666667, 0.57142857, 0.5       ])

In [17]:
expected_abd=np.array([0, 0.4, 0.3, 0.2, 0.1, 0])
observed_abd=np.array([0.1, 0, 0.2, 0.7, 0, 0])
updated_prc(expected_abd, observed_abd)

array([0.66666667, 1.        , 0.66666667, 0.5       , 0.5       ,
       0.5       , 0.57142857, 0.66666667, 0.57142857, 0.5       ])

## load the data table

In [18]:
file="D_M_K_m_Raw_AbdTable_1225.txt"

In [25]:
data=pd.read_csv(file, sep='\t', index_col=0)

In [24]:
data[data['Environment']=='Building1']

,Tools,Environment,taxid_39491,taxid_28118,taxid_214856,taxid_43675,taxid_47678,taxid_823,taxid_1328,taxid_755731,...,taxid_123456889,taxid_123456890,taxid_123456891,taxid_123456892,taxid_123456893,taxid_123456894,taxid_123456895,taxid_123456896,taxid_123456897,taxid_123456898
SampleID,,,,,,,,,,,,,,,,,,,,,
truth_tax_building1,Truth_tax,Building1,0.009656,0.0,0.0,0.026999,0.003488,0.0,0.003738,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
truth_seq_building1,Truth_seq,Building1,0.011772,0.0,0.0,0.021606,0.005634,0.0,0.002950,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
building1_kraken2,Kraken2,Building1,0.011500,0.0,0.0,0.021500,0.005500,0.0,0.002000,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
building1_bracken,Bracken,Building1,0.012100,0.0,0.0,0.022100,0.005800,0.0,0.002900,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
building1_MPA2,MPA2,Building1,0.011538,0.0,0.0,0.028271,0.004278,0.0,0.003938,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
building1_mOTUs2,mOTUs2,Building1,0.008785,0.0,0.0,0.028460,0.001635,0.0,0.003664,0.0,...,0.003856,0.0,0.0,0.003178,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
ids = data.Environment.unique().tolist()
n_ids = len(ids)
#abd_type = data.abd_type.unique().tolist()
#obs = data.obs.unique().tolist()

### calculate the AUPR for four profilers on a single sample

In [100]:
tmp=data[data['Environment']=='Building1']
for i in range(2, 6):
    print(updated_prc(tmp.iloc[0, 3:], tmp.iloc[i, 3:]))
    print(updated_prc(tmp.iloc[1, 3:], tmp.iloc[i, 3:]))

[0.69426752 1.         0.97979798 1.         0.00917431 0.88990826
 0.81954887 0.01818182 0.93269231 0.95696266]
[0.69426752 1.         0.97979798 1.         0.00917431 0.88990826
 0.81954887 0.01818182 0.93269231 0.95696266]
[0.6300578  1.         0.97058824 1.         0.00917431 0.90825688
 0.77304965 0.01818182 0.93838863 0.95347554]
[0.6300578  1.         0.97058824 1.         0.00917431 0.90825688
 0.77304965 0.01818182 0.93838863 0.95347554]
[0.90082645 0.         0.99056604 1.         0.         0.96330275
 0.94782609 0.         0.97674419 0.97655042]
[0.90082645 0.         0.99056604 1.         0.         0.96330275
 0.94782609 0.         0.97674419 0.97655042]
[0.93043478 1.         0.94339623 0.98165138 0.00917431 0.91743119
 0.95535714 0.01818182 0.93023256 0.96326435]
[0.93043478 1.         0.94339623 0.98165138 0.00917431 0.91743119
 0.95535714 0.01818182 0.93023256 0.96326435]


/Users/huangshi/anaconda3/envs/qiime2-2020.2/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### calculate the AUPR for four profilers for all 25 samples

In [92]:
thresholds = np.concatenate((np.arange(0.0, 0.0001, 0.00001), 
                             np.arange(0.0001, 0.001, 0.0001), 
                             np.arange(0.001, 0.011, 0.001))).tolist()

thresholds_str = ['{:.5f}'.format(x) for x in thresholds]
thresholds_num =[float(x) for x in thresholds_str]

In [94]:
metrics = ['precision', 'recall', 'f1']
metrics_rep = [item for item in metrics for i in range(len(thresholds))]
thresholds3 = thresholds_str*len(thresholds)
concat_func = lambda x,y: x + "_" + str(y)

In [99]:
name_all = list(map(concat_func, metrics_rep,  thresholds3))
name_all.append('aupr')
tax_name_all = list(map(concat_func, ["tax"]*len(name_all), name_all))
seq_name_all = list(map(concat_func, ["seq"]*len(name_all), name_all))
colnames = tax_name_all + seq_name_all
#colnames

In [96]:
ncols=(len(thresholds_num)*3+1)*2
out = []
rownames = []
for i in ids: #xrange(len(ids)):
    tmp=data.groupby('Environment').get_group(i)
    rownames.extend(tmp.index.values[2:].tolist())
    #sample_arr = np.empty([4, ncols])
    for j in range(2, 6):
        tax = updated_prc(tmp.iloc[0, 2:], tmp.iloc[j, 2:], thresholds_num)
        seq = updated_prc(tmp.iloc[1, 2:], tmp.iloc[j, 2:], thresholds_num)
        out.append(np.concatenate((tax, seq)))
np_out=np.array(out)


In [97]:
out=pd.DataFrame(data=np_out, columns=colnames, index=rownames)
out

,tax_precision_0.00000,tax_precision_0.00001,tax_precision_0.00002,tax_precision_0.00003,tax_precision_0.00004,tax_precision_0.00005,tax_precision_0.00006,tax_precision_0.00007,tax_precision_0.00008,tax_precision_0.00009,...,seq_f1_0.00200,seq_f1_0.00300,seq_f1_0.00400,seq_f1_0.00500,seq_f1_0.00600,seq_f1_0.00700,seq_f1_0.00800,seq_f1_0.00900,seq_f1_0.01000,seq_aupr
building1_kraken2,0.696203,0.696203,0.696203,0.696203,0.696203,0.696203,0.696203,0.696203,0.696203,0.696203,...,0.845361,0.773481,0.629630,0.539474,0.479452,0.414286,0.379562,0.367647,0.318182,0.957327
building1_bracken,0.632184,0.632184,0.632184,0.632184,0.632184,0.632184,0.632184,0.632184,0.632184,0.632184,...,0.880000,0.782609,0.666667,0.594937,0.500000,0.479452,0.402878,0.391304,0.355556,0.953918
building1_MPA2,0.901639,0.909091,0.916667,0.924370,0.940171,0.940171,0.940171,0.940171,0.948276,0.948276,...,0.932692,0.837696,0.738636,0.646341,0.621118,0.539474,0.489796,0.447552,0.436620,0.976681
building1_mOTUs2,0.931034,0.931034,0.931034,0.931034,0.931034,0.931034,0.931034,0.931034,0.931034,0.931034,...,0.893204,0.791444,0.701754,0.633540,0.580645,0.523490,0.472222,0.439716,0.417266,0.963804
building2_kraken2,0.818182,0.818182,0.818182,0.818182,0.818182,0.818182,0.818182,0.818182,0.818182,0.818182,...,0.883436,0.794702,0.750000,0.695652,0.656716,0.582677,0.560000,0.512397,0.500000,0.982003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
vaginal4_mOTUs2,0.903226,0.903226,0.903226,0.903226,0.903226,0.903226,0.903226,0.903226,0.903226,0.903226,...,1.000000,1.000000,0.981818,0.962963,0.962963,0.901961,0.857143,0.857143,0.833333,0.964286
vaginal5_kraken2,0.360000,0.360000,0.360000,0.360000,0.360000,0.360000,0.360000,0.360000,0.360000,0.360000,...,0.981818,0.962963,0.962963,0.943396,0.880000,0.897959,0.875000,0.826087,0.826087,0.957023
vaginal5_bracken,0.306818,0.306818,0.306818,0.306818,0.306818,0.306818,0.306818,0.306818,0.306818,0.306818,...,0.931034,0.964286,0.981818,0.981818,0.923077,0.901961,0.901961,0.833333,0.851064,0.953652
vaginal5_MPA2,0.843750,0.931034,0.964286,0.964286,0.964286,0.964286,0.964286,0.964286,0.964286,0.964286,...,0.981818,0.981818,0.981818,0.981818,0.981818,0.962963,0.943396,0.923077,0.923077,0.929685


In [98]:
out.to_csv('all_raw_PRanalysis_out_{}_to_{}.{}.tsv'.format(str(min(thresholds_num)), str(max(thresholds_num)), str(len(thresholds_num))), sep='\t')